<a href="https://colab.research.google.com/github/Tracari/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 http:

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://software-reviews.s3.amazonaws.com/amazon_reviews_us_Software_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [ ]:
# Create Vine_df using columns: review_id, star_rating, helpful_votes, total_votes, vine, verified_purchase
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|                Y|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|                Y|
|R1XU1B93402SYJ|          1|            1|          1|   N|                Y|
|R2U432NB3OPVR0|          5|            0|          0|   N|                Y|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|     

In [ ]:
# filter df for total_votes count equal to or greater than 20

total_votes_df = df.filter(df.total_votes >= 20)
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   31044774|R26CC1STPSUHBS|B01019T6O0|     510976915|Microsoft Windows...|        Software|          1|           27|         39|   N|                N|Do NOT take the "...|Do NOT take the &...| 2015-08-31|
|         US|   51362864| R7K83D59T9KWS|B00ZSI7Y3U|     604043733|Microsoft Windows...|        Software|          1|    

In [ ]:
# Create a new df where all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

greater_fifty = total_votes_df.filter(total_votes_df["helpful_votes"] / total_votes_df["total_votes"] >= .5)
greater_fifty.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   31044774|R26CC1STPSUHBS|B01019T6O0|     510976915|Microsoft Windows...|        Software|          1|           27|         39|   N|                N|Do NOT take the "...|Do NOT take the &...| 2015-08-31|
|         US|   51362864| R7K83D59T9KWS|B00ZSI7Y3U|     604043733|Microsoft Windows...|        Software|          1|    

In [ ]:
# Import the sum function 
from pyspark.sql.functions import sum

# Confirm total number of reviews where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

greater_fifty_total = greater_fifty.agg({"total_votes": "sum"})
greater_fifty_total.show()

+----------------+
|sum(total_votes)|
+----------------+
|          918702|
+----------------+



In [ ]:
# Create a df where all the rows where a review was written as part of the Vine program (paid)

paid_vine = greater_fifty.filter(greater_fifty.vine == "Y")
paid_vine.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12051324|R28EIR3Z6RRUNN|B00UV3VCLC|     617825071|Cyberlink PowerDV...|        Software|          2|           50|         60|   Y|                N|Frustrating, but ...|My computer has a...| 2015-06-11|
|         US|   50867647|R2YO2RESXZEWOF|B00T0I6J9E|     636727185|        Corel Parent|        Software|          4|    

In [ ]:
# Create a df where all the rows where a review was written was not part of the Vine program (unpaid)

unpaid_vine = greater_fifty.filter(greater_fifty.vine == "N")
unpaid_vine.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   31044774|R26CC1STPSUHBS|B01019T6O0|     510976915|Microsoft Windows...|        Software|          1|           27|         39|   N|                N|Do NOT take the "...|Do NOT take the &...| 2015-08-31|
|         US|   51362864| R7K83D59T9KWS|B00ZSI7Y3U|     604043733|Microsoft Windows...|        Software|          1|    

In [ ]:
# Total number of paid reveiws

# dataframe.agg({'column_name': 'sum'})
#total_paid = paid_vine.agg({"total_votes": "sum"})
#total_paid.show()

total_paid = paid_vine.select(sum("total_votes"))
total_paid.show()



+----------------+
|sum(total_votes)|
+----------------+
|           20314|
+----------------+



In [ ]:
# total number of 5 star reviews of paid reviews
five_star_paid = paid_vine.filter(paid_vine["star_rating"] == "5").agg({'total_votes':"sum"})
five_star_paid.show()



+----------------+
|sum(total_votes)|
+----------------+
|            8079|
+----------------+



In [ ]:
# percentage of 5 star reviews of paid
five_star_pct = 8079 / 20314
five_star_pct

0.39770601555577434

In [ ]:
# Total number of unpaid reviews

total_unpaid = unpaid_vine.agg({"total_votes": "sum"})
total_unpaid.show()

# dataframe.agg({'column_name': 'sum'})

+----------------+
|sum(total_votes)|
+----------------+
|          898388|
+----------------+



In [ ]:
# total number of 5 star reviews of unpaid reviews
five_star_unpaid = unpaid_vine.filter(unpaid_vine["star_rating"] == "5").agg({"total_votes":"sum"})
five_star_unpaid.show()

+----------------+
|sum(total_votes)|
+----------------+
|          267089|
+----------------+



In [ ]:
# percentage of 5 star reviews of paid
five_star_pct_unpaid = 267089 / 898388
five_star_pct_unpaid

0.2972980493951389

In [ ]:
################_________________Additional_Vine_Review_Analysis_______________

In [ ]:
# Total 1-2 stars reviews for paid vine

one_two_paid = paid_vine.filter(paid_vine["star_rating"] <= "2")
one_two_paid.agg({"total_votes":"sum"}).show()



+----------------+
|sum(total_votes)|
+----------------+
|            3309|
+----------------+



In [ ]:
# Total 1-2 stars reviews for unpaid vine

one_two_unpaid = unpaid_vine.filter(unpaid_vine["star_rating"] <= "2")
one_two_unpaid.agg({"total_votes":"sum"}).show()


+----------------+
|sum(total_votes)|
+----------------+
|          417980|
+----------------+



In [ ]:
# Percentage of 1-2 star reviews for paid vine

negative_paid = 3309/ 20314
negative_paid


0.16289258639362017

In [ ]:
# Percentage of 1-2 star reviews for unpaid vine

negative_unpaid = 417980/898388
negative_unpaid


0.4652555466012458

In [ ]:
# Total 4-5 stars reviews for paid vine

four_five_paid = paid_vine.filter(paid_vine["star_rating"] >= "4")
four_five_paid.agg({"total_votes":"sum"}).show()

+----------------+
|sum(total_votes)|
+----------------+
|           12768|
+----------------+



In [ ]:
# Total 4-5 stars reviews for unpaid vine

four_five_unpaid = unpaid_vine.filter(unpaid_vine["star_rating"] >= "4")
four_five_unpaid.agg({"total_votes":"sum"}).show()

+----------------+
|sum(total_votes)|
+----------------+
|          390116|
+----------------+



In [ ]:
# Percentage of 4-5 star reviews for paid vine

four_five_pct = 12768/ 20314
four_five_pct

0.6285320468642316

In [ ]:
# Percentage of 4-5 star reviews for unpaid vine

four_five_pct_unpaid = 390116/898388
four_five_pct_unpaid


0.4342399943009034

In [ ]:
dept = [("5-stars", five_star_pct, five_star_pct_unpaid),
        ("4-5 stars", four_five_pct, four_five_pct_unpaid),
        ("1-2 stars", negative_paid, negative_unpaid)]

deptColumns = ["Rating", "Paid", "Unpaid"]

In [ ]:
reviews_df = spark.createDataFrame(data=dept, schema = deptColumns)
reviews_df.printSchema()
reviews_df.show(truncate=False)



root
 |-- Rating: string (nullable = true)
 |-- Paid: double (nullable = true)
 |-- Unpaid: double (nullable = true)

+---------+-------------------+------------------+
|Rating   |Paid               |Unpaid            |
+---------+-------------------+------------------+
|5-stars  |0.39770601555577434|0.2972980493951389|
|4-5 stars|0.6285320468642316 |0.4342399943009034|
|1-2 stars|0.16289258639362017|0.4652555466012458|
+---------+-------------------+------------------+



In [ ]:
from pyspark.sql.functions import format_number
from pyspark.sql.functions import concat,col,lit
reviews_df.withColumn("Paid",concat((col("Paid") * 100).cast("int"),lit('%')))
reviews_df.withColumn("Unpais",concat((col("Paid") * 100).cast("int"),lit('%'))).show()

+-------------------+------------------------------------+
|               Paid|concat(CAST((Paid * 100) AS INT), %)|
+-------------------+------------------------------------+
|0.39770601555577434|                                 39%|
| 0.6285320468642316|                                 62%|
|0.16289258639362017|                                 16%|
+-------------------+------------------------------------+

